# Prosessering av Energy Dispersive X-ray Spectroscopy data

Denne Jupyter Notebooken viser hvordan Energy Dispersive X-ray Spectroscopy (EDS) data kan analyseres. Spektroskopi er en veldig viktig datatype "klasse", som dukker opp i mange forskjellige teknikker.

### Målet med denne notebooken

- Dere skal kunne prosessere EDS dataene dere tok opp i SEM-laben
- Bli komfortable med å jobbe med spektroskopidata
- Lage figur som viser kjemisk komposisjon i dataene deres

### Notebook-planen

- Åpne datasettet, og utforske det
- Finne ut hvilke grunnstoffer som er i prøvematerialet
- Lage figur av dette

## Importere biblioteker

Først, plotte-biblioteket. Dette kan enten være `%matplotlib qt` for egne vinduer for plottene, eller `%matplotlib widget` for å få plottene i selve Jupyter Notebooken.

In [ ]:
%matplotlib qt
from hyperspy import api as hs

Så importere HyperSpy

## Åpne dataset

Dette gjøres via `hs.load`, som kan åpne en rekke dataformater, spesielt innenfor elektronmikroskopi. Velg et av EDS datasettene deres, disse skal ha:

- `.hspy` filformat
- Ha filnavn som inneholder noe med: `EDS` eller `EDX`

Lage et objekt som heter `s`

In [ ]:
s = hs.load("data/hdf5-files/Map Data 3.hdf5")

Så skriv `s` i cellen under, og kjør den

Her ser vi et par viktige ting: signalet er et `EDSSEMSpectrum`, og den har 3 dimensjoner! 
    
Dimensjonene ser vi helt i slutten, som har 3 tall (som nok vil være annerledes for deres datasett): `(512, 352|1024)`. Tallene til venstre for `|` er navigasjonsdimensjonene, mens tallet til høyre for `|` er signaldimensjonen: `(NAVIGASJON 0, NAVIGASJON 1|SIGNAL 1)`.

I denne datatypen, så er de 2 navigasjonsdimensjonene probe-posisjonen, og signaldimensjonen er energien til røntgenstrålene.

Dette betyr at signaldimensjonen er 1-dimensjonal, som stemmer bra med at dette er spektroskopisk data.

(Hvis signalet IKKE er `EDSSEMSpectrum`, så kjør `s.set_signal_type("EDS_SEM")`.

## Enkel utforskning av dataene

Nå kan vi visualisere dataene, og se hvordan spektrumene ser ut. Bruk `plot` funksjonen i `s`, og utforsk datasettet.

<img src="figurer/sem_eds_navigator.jpg" width=900 height=900 />

In [ ]:
#s.plot()

Her ser vi at hver probe-posisjon har veldig få røntgen-tellinger, ergo at signalet ikke er særlig bra. Noen steder kan vi se at det er klare topper, men disse er for det meste under 10 tellinger.

Dette skal vi gjøre noe med, men først vil vi utforske datasettet som funksjon av røntgenstråle-energien. Bruk transpose funksjonaliteten i s, og lag et nytt signal `st`: `st = s.T`

In [ ]:
st = s.T

Skriv `st` i cellen under, og kjør den.

Nå er signalet `Signal2D`: navigasjon- og signal-dimensjonene har blitt byttet om. Så nå kan vi navigere over datasettet som en funksjon av røntgen-energien, istedet for probe-posisjonen.

Så plot `st`. Merk at nå er navigatoren i "røntgen" plottet.

<img src="figurer/eds_transpose.jpg" width=900 height=900 />

Flytt navigatoren frem og tilbake, spesielt på de klare toppene, og se om forskjelliges steder på prøven lyser opp.

In [ ]:
#st.plot()

Nå er røntgen-tellingene mye høyere, siden det er en sum av alle probe-posisjonene.

Så vi ser at det er noe interessant i dataene. Det neste steget er å lage bilder som viser hvor de forskjellige grunnstoffene er.

## Mer avansert

### Finne grunnstoffene

Det første vi må gjøre her, er å finne ut hvilke grunnstoffer vi har i prøvematerialet.

Enkleste måten å gjøre dette på, er å se på toppene vi har røntgen-signalet, kombinert med det vi vet om prøvematerialet.

Så: summer opp alle probeposisjonene, til et røntgen-energi signal. Bruk `sum` funksjonen i `s` til å lage et nytt signal `s_sum`.

In [ ]:
s_sum = s.sum()

Så bruk `plot` funksjonen i `s_sum` til å visualisere dette signal, og finn ut hva slags grunnstoffer vi har.

<img src="figurer/eds_sum.jpg" width=900 height=900 />

Dere kan se hvilke topper de forskjellige grunnstoffene lager, ved for eksempel å bruke denne: https://www.jeolusa.com/DesktopModules/LiveContent/API/Image/Get?mid=4725&eid=1&Type=View&PortalId=2

In [ ]:
#s_sum.plot()

**Gå igjennom alle toppene, og prøv å finn ut hvilket grunnstoff de tilhører.**

Merk at samme grunnstoff kan ha flere topper, så hvis dere er usikre så er et triks å sjekke om de andre toppene også er med spektrumet.

- 9.4 -> Pt (svakt)
- 7.46 -> Ni
- 6.39 -> Fe
- 4.65 -> La*
- 4.02 -> Xe / Sc*
- 3.69 -> Ca*
- 3.44 -> Sn*
- 2.83 -> Pd*
- 2.30 -> S*
- 2.05 -> Pt
- 2.00 -> P*
- 1.74 -> Si (*)
- 1.48 -> Al (?) Br (?) (*)
- 0.84 -> Ni (*)
- 0.70 -> Fe (*)
- 0.52 -> O (*)
- 0.38 -> N (*) 
- 0.26 -> C (*)

### Legge til grunnstoffene i signalet

Nå som dere har funnet grunnstoffene, så må de legges til i `s` signalet.

Dette gjøres via `set_elements` funksjonen som er i `s`. Parameter som skal til `set_elements` må være en liste, og hvert grunnstoff må være i formen `"Si"`, `"C"`, ...

Tips: se i docstring til `set_elements` via "Shift + Tab" på tastaturet.

In [ ]:
#elements = ["Pt", "Ni", "Fe", "Si", "O", "N", "C", "Al"]
elements =  ["Al", "C", "Ca", "N", "Ni", "O", "S", "Si"] 
elements.sort()
s.set_elements(elements)

Sjekk at alt har blitt lagt til, via `metadata.Sample` attribute i `s`

Så legger vi til røntgen linjene til disse grunnstoffene, via `add_lines` i `s`

In [ ]:
s.add_lines()

Så se hva som har blitt lagt til i metadataen, via `metadata.Sample` attribute i `s`

In [ ]:
s.metadata.Sample

Her ser dere at det bare har blitt lagt til en linje per grunnstoff. Dette er fordi den med lavest energi er det mest relevant.

Dette kan dere så plotte, ved å bruke `plot` med `xray_lines=True` argumentet

In [ ]:
#s.plot(xray_lines=True)

Så kan vi hente ut intensiteten for alle disse linjene, over hele datasettet.

Til dette bruker vi `get_lines_intensity` som er i `s`. Lagre resultatet til dette i en ny variabel: `linjer`. I tillegg, så bruk `plot_results=True` i `get_lines_intensity`. Dette vil åpne en plotte-vindu for hvert grunnstoff.

In [ ]:
linjer = s.get_lines_intensity()#plot_result=True)

Så kan vi se hva som er i `linjer` variablen. Skriv `linjer` i cellen under, og kjør den.

#### Sjekke integrasjonsvinduet

Her ser vi at den er en liste med signaler, et for hvert grunnstoff. For å plotte dem: `linjer[0].plot()`

Et mulig problem med denne typen prosessering, er hvis røntgen-linjene er så nærme at de overlapper. En måte å sjekke dette på, er å se hvor "bredde" integrasjonsvinduene er.

Dette gjøres enkelts med å først å summere datasettet igjen. Bruk `sum` via `s`, til å lage en ny variabel `s_sum2`.

Så `plot` `s_sum2`, med argumentet `integration_windows='auto'`. Bruk forstørrelse-funksjonen til å sjekke at det ikke er for mye overlapp.

In [ ]:
s_sum2 = s.sum()

In [ ]:
#s_sum2.plot(integration_windows="auto")

### Lage bilder av hvor grunnstoffene er

Nå som vi kan se hvor de forskjellige grunnstoffene er, så lager vi en figur som viser dette.

Først henter vi ut et og et grunnstoff, kall disse `s_si`, `s_fe`, ... . Siden `linjer` er en liste, så gjøres dette med `linjer[0]`, `linjer[1]`, ... . Pass på å sjekke hvilket grunnstoff de forskjellige signalene er!

Viktig: disse signalene må transposes! Så kommandoen blir `linjer[0].T`.

Så kan vi lage en matplotlib figur med et subplot per grunnstoffer dere har.

- Tips 1: har dere backscatter elektron eller Sekundærelektron bilde, så er det også fint å ta med! Da må dere ha et ekstra subplot i tillegg.
- Tips 2: hvis dere har veldig mange grunnstoff, så kan den være en fordel å ha 2 vertikale rader med subplot.
- Tips 3: bruk figsize parameteren til å lage figures større, slik at subplotene passer inn. F.eks. hvis dere har 4 horisontale subplot, så kan `figsize=(20, 5)` passe bra.

Importer matplotlib, lag figur og subplot objekter.

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import matplotlib.patheffects as patheffects

In [ ]:
plt.figure()

fontprops = fm.FontProperties(size=10)
scalebar_kwargs = {
    'size': 1, 
    'label': '1 µm', 
    'loc': 4, 
    'frameon': False, 
    'color': 'white', 
    'size_vertical': 0.1, 
    'label_top': False, 
    'fontproperties': fontprops
    }

for i in range(8):
    t = elements[i]
    d = linjer[i]
    d = d.T
    plt.subplot(2, 4, i+1)
    plt.imshow(d, extent = d.axes_manager.signal_extent)
    plt.title(t)
    plt.colorbar()
    plt.xticks([])
    plt.yticks([])
    scalebar = AnchoredSizeBar(transform=plt.gca().transData, **scalebar_kwargs)
    scalebar.txt_label._text.set_path_effects([patheffects.withStroke(linewidth=2, foreground='black', capstyle="round")])
    plt.gca().add_artist(scalebar)

plt.show()

Så bruk `imshow` med forskjellige fargekart (`cmap`) til å visualisere dataene: for eksempel `"Blues_r"`, `"Greens_r"`, ... Se [matplotlib dokumentasjonen](https://matplotlib.org/stable/tutorials/colors/colormaps.html) for en fullstendig liste over alle fargekartene.

Et ekstra plotte-element som burde være med her er en ting som viser antall tellinger. Dette gjøres ved å:

- `cax_si = ax_si.imshow(.....)`
- `fig.colorbar(cax_si, ax=ax_si)`

Legge til colorbars

Så bruk tingene dere har lært i de andre dataøvingen, til å lage en figur:

- Legg til scalebar
- Ha annoteringer (a, b, c, ...), skriv hvilket grunnstoff det er: "Fe", "Si", ...
- Fjern tomrommet som kommer rundt plottene
- Fjern tallene som er rundt bildene. Her må dere mest sannsynlig manuelt stille på `figsize` til det blir bra

Eksempel på en sånn figur, men uten alle annoteringene

<img src="figurer/eds_kart.jpg" width=900 height=900 />